# Classifier for Elephants:  African vs. Asian
## about 200 in each class
### Nov 5, 2018

In [ ]:
import fastai
fastai.show_install(0)

In [ ]:
!pwd

# Creating your own dataset from Google Images

*by: Francisco Ingham and Jeremy Howard. Inspired by [Adrian Rosebrock](https://www.pyimagesearch.com/2017/12/04/how-to-create-a-deep-learning-dataset-using-google-images/)*

In this tutorial we will see how to easily create an image dataset through Google Images. **Note**: You will have to repeat these steps for any new category you want to Google (e.g once for dogs and once for cats).

## Get a list of URLs

### Search and scroll

Go to [Google Images](http://images.google.com) and search for the images you are interested in. The more specific you are in your Google Search, the better the results and the less manual pruning you will have to do.

Scroll down until you see a button that says 'Show more results'. All the images you scrolled past are now available to download. To get more, click on the button. Then continue scrolling until you cannot scroll anymore. The maximum number of images Google Images shows is 700.

### Download into file

Now you must run some Javascript code in your browser which will save the URLs of all the images you want for you dataset.

Press <kbd>Ctrl</kbd><kbd>Shift</kbd><kbd>J</kbd> in Windows/Linux and <kbd>Cmd</kbd><kbd>Opt</kbd><kbd>J</kbd> in Mac, and a small window the javascript 'Console' will appear. That is where you will paste the JavaScript commands.

You will need to get the urls of each of the images. You can do this by running the following commands:

```javascript
urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```

### Create directory and upload urls file into your server

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
import glob
from io import BytesIO
import numpy as np

In [4]:
project = "elphants"

In [5]:
my_path = "/home/jupyter/tutorials/"
path = Path(my_path+'data/'+project)

---

### Setting up image folders
Choose an appropriate name for your labeled images. You can run these steps multiple times to grab different labels.

In [ ]:
num_images=200

In [ ]:
folder = 'african'
file = 'urls_african.txt'

In [ ]:
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [ ]:
folder = 'asian'
file = 'urls_asian.txt'

In [ ]:
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

Finally, upload your urls file. You just need to press Upload in your working directory and select your file, then click 'upload' on the right.

![](images/download_images/upload.png)

## Download images

Now you will need to download you images from their respective urls.

fast.ai has a function that allows you to do just that. You just have to specify the urls filename and the destination folder and this function will download and save all images than can be opened. If they have some problem in being opened, they will not be saved.

Let's download our images! Notice you can choose a maximum number of images to be downloaded. In this case we will not download all the urls.

In [ ]:
!head {my_path}data/elephants/urls_african.txt

In [ ]:
folder = 'african'
file = 'urls_african.txt'

path = Path(my_path+'data/elephants')

dest = path/folder

download_images(path/file, dest, max_pics=num_images)

In [ ]:
folder = 'asian'
file = 'urls_asian.txt'

path = Path(my_path+'data/elephants')

dest = path/folder

download_images(path/file, dest, max_pics=num_images)

## Check how many images in each class

In [ ]:
!ls /home/jupyter/tutorials/data/elephants/african | wc -l

In [ ]:
!ls /home/jupyter/tutorials/data/elephants/asian | wc -l

## Verify images

Good! Let's take a look at some of our pictures then.

In [ ]:
classes = ['african','asian']

In [ ]:
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_workers=8)

In [ ]:
!ls /home/jupyter/tutorials/data/elephants/african | wc -l

In [ ]:
!ls /home/jupyter/tutorials/data/elephants/asian | wc -l

## Training: resnet34

In [ ]:
??ImageDataBunch

In [ ]:
np.random.seed(42)
bs=64
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2, ds_tfms=get_transforms(), size=224, \
                                  num_workers=2, bs=bs)

In [ ]:
(path).ls()

### View data

In [ ]:
data.show_batch(rows=2, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)  

In [ ]:
learn = learner.create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
#learn.fit_one_cycle(6, max_lr=0.5)
#learn.fit_one_cycle(6, max_lr=0.25)
#learn.fit_one_cycle(6, max_lr=0.05)
#learn.fit_one_cycle(6, max_lr=0.025)
#learn.fit_one_cycle(6, max_lr=0.01)
learn.fit_one_cycle(6, max_lr=0.001)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
# if plot is blank
learn.recorder.plot(skip_start=0, skip_end=0)

In [ ]:
learn.save('resnet34-stage-1')

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(1e-6,3e-5))

In [ ]:
learn.save('resnet34-stage-2')

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

What the numbers represent:  
prediction | actual | loss | probability it was predicted

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
doc(interp.plot_top_losses)

In [ ]:
interp.plot_confusion_matrix(figsize=(4,4), dpi=100)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
interp.most_confused(min_val=0)


## Unfreezing, fine-tuning, and learning rates

Since our model is working as we expect it to, we will *unfreeze* our model and train some more.

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load('resnet34-stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(1e-6,1e-4))

That's a pretty accurate model!

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(4, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(4,4), dpi=90)

---

## Training: resnet50

Now we will train in the same way as before but with one caveat: instead of using resnet34 as our backbone we will use resnet50 (resnet34 is a 34 layer residual network while resnet50 has 50 layers. Later in the course you can learn the details in the [resnet paper](https://arxiv.org/pdf/1512.03385.pdf)).

Basically, resnet50 usually performs better because it is a deeper network with more parameters. Let's see if we can achieve a higher performance here. To help it along, let's us larger images too, since that way the network can see more detail. We reduce the batch size a bit since otherwise this larger network might eat up our GPU memory.

In [16]:
np.random.seed(42)
bs=16

In [17]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2, ds_tfms=get_transforms(), size=224, \
                                  num_workers=1, bs=bs)

ValueError: num_samples should be a positive integeral value, but got num_samples=0

### View data

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

### Learner

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(6)
#learn.fit_one_cycle(6, max_lr=0.03)

## Unfreezing, fine-tuning, and learning rates

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
# if plot is blank
learn.recorder.plot(skip_start=0, skip_end=0)

In [ ]:
learn.save('resnet50-stage1')

In [ ]:
learn.fit_one_cycle(4, max_lr=slice(3e-7,3e-6))

In [ ]:
learn.load('resnet50-stage1')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('resnet50-stage2')

In this case it doesn't, so let's go back to our previous model.

In [ ]:
#learn.load('resnet50-stage-1')

## Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=0)

## Save final model

In [ ]:
final_model_name = 'resnet50-stage2'

## Predicting for New Images

In [ ]:
model_path="/home/jupyter/tutorials/data/elephants/models"

In [ ]:
learn.load(final_model_name)

In [ ]:
??create_cnn

In [ ]:
data2 = ImageDataBunch.single_from_classes("", 
                                           learn.data.classes,
                                          tfms=get_transforms(),
                                          size=224).normalize(imagenet_stats)
learn2 = create_cnn(data2, models.resnet50)
#learn2.load('resnet50-stage2')

In [ ]:
data2.classes, data2.c

In [ ]:
# let's test my photo
url ="https://github.com/reshamas/reshamas.github.io/blob/master/assets/images/fl1_rs_camel.jpg?raw=true"

In [ ]:
url = "https://media.gettyimages.com/photos/bo-derek-attends-sicab-2012-the-international-horse-fair-of-spain-on-picture-id156965178"

In [ ]:
response = requests.get(url)
img = open_image(BytesIO(response.content))
img

In [ ]:
pred_class, pred_idx, outputs = learn2.predict(img)

In [ ]:
learn2.predict(img)

In [ ]:
print(pred_class, pred_idx, outputs, outputs[pred_idx])